# Použité knihovny

In [ ]:
import wfdb
import os
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import butter, filtfilt


In [ ]:
# Hlavní adresář se složkami EKG měření
main_directory = "../data/ekg"

# Načtení seznamu složek
folders = sorted([f for f in os.listdir(main_directory) if os.path.isdir(os.path.join(main_directory, f))])
print(f"Nalezené složky: {folders}")

In [ ]:
lowcut = 0.5  # Dolní hranice pásmové filtrace (Hz)
highcut = 50.0  # Horní hranice pásmové filtrace (Hz)
filter_order = 4  # Pořadí filtru
threshold = 0.5  # Práh pro detekci R vrcholů

In [2]:



# Uložení výsledků BPM pro každé měření
bpm_results = []

for folder in folders:
    ekg_directory = os.path.join(main_directory, folder)

    # Načtení všech dostupných souborů ve složce
    files = os.listdir(ekg_directory)
    signal_basenames = set(f.split(".")[0] for f in files if f.endswith("ECG.dat"))

    if len(signal_basenames) > 0:
        signal_basename = list(signal_basenames)[0]
        record_path = os.path.join(ekg_directory, signal_basename)

        # Načtení signálu
        record = wfdb.rdsamp(record_path)
        signal = record[0][:, 0]  # První kanál
        fields = record[1]

        # Parametry
        sampling_frequency = fields["fs"]  # Vzorkovací frekvence (Hz)
        

        # Normalizace signálu
        signal_normalized = (signal - np.mean(signal)) / np.std(signal)

        # Funkce pro pásmovou filtrační metodu
        def bandpass_filter(signal, lowcut, highcut, fs, order):
            nyquist = 0.5 * fs
            low = lowcut / nyquist
            high = highcut / nyquist
            b, a = butter(order, [low, high], btype="band")
            return filtfilt(b, a, signal)

        # Aplikace filtrace
        try:
            signal_processed = bandpass_filter(signal_normalized, lowcut, highcut, sampling_frequency, filter_order)
        except ValueError as e:
            print(f"Chyba filtrace: {e}")
            signal_processed = signal_normalized

        # Derivace signálu
        derivative = np.gradient(signal_processed)

        # Hledání nulových průchodů podle derivace
        zero_crossings = np.where((derivative[:-1] > 0) & (derivative[1:] < 0))[0]

        # Filtrování nulových průchodů podle amplitudy (práh)
        peaks = zero_crossings[signal_processed[zero_crossings] > threshold]

        # Zajištění stejného počtu vzorků pro signal_processed a detekované vrcholy
        peaks = peaks[peaks < len(signal_processed)]

        # Výpočet tepové frekvence
        if len(peaks) > 1:
            r_peak_intervals = np.diff(peaks) / sampling_frequency  # Intervaly mezi R vrcholy
            heart_rate = 60 / np.mean(r_peak_intervals) if len(r_peak_intervals) > 0 else 0
        else:
            heart_rate = 0

        print(f"Složka: {folder}, Počet detekovaných R vrcholů: {len(peaks)}, Průměrná tepová frekvence: {heart_rate:.2f} BPM")

        # Uložení výsledků BPM
        bpm_results.append((folder, heart_rate))

# Vykreslení grafu
folders, bpm_values = zip(*bpm_results)
plt.figure(figsize=(12, 6))
plt.bar(folders, bpm_values, color="skyblue")
plt.title("Tepová frekvence (BPM) pro jednotlivá měření")
plt.xlabel("Měření")
plt.ylabel("BPM")
plt.xticks(rotation=45)
plt.grid(axis="y")
plt.show()


Nalezené složky: ['100001', '100002', '103001', '103002', '103003', '104001', '105001', '111001', '113001', '114001', '115001', '118001', '121001', '122001', '123001', '124001', '125001', '126001']


KeyboardInterrupt: 